In [2]:
import os
from dotenv import load_dotenv

import pandas as pd
from langchain.chat_models import init_chat_model
from langchain.tools import tool
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_huggingface import HuggingFaceEmbeddings

load_dotenv() 

d:\nlp\Anime\anienv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [3]:
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY

In [4]:
import os 
from langchain.chat_models import init_chat_model

model = init_chat_model("groq:llama-3.3-70b-versatile")
response = model.invoke("Why am I studying at DOT pune")
response

AIMessage(content="It seems like you're studying at the Design Orbit and Technology (DOT) institute in Pune, India. \n\nDOT Pune is a popular institute that offers various courses in design, technology, and management. Here are some possible reasons why you might be studying at DOT Pune:\n\n1. **Quality Education**: DOT Pune is known for its high-quality education and experienced faculty. You might be studying there to gain a strong foundation in your chosen field.\n2. **Career Opportunities**: The institute has a strong industry connection, which can lead to good job opportunities after graduation. You might be studying at DOT Pune to enhance your career prospects.\n3. **Specialized Courses**: DOT Pune offers a range of specialized courses that might not be available at other institutes. You might be studying there to gain expertise in a specific area, such as design, technology, or management.\n4. **State-of-the-Art Infrastructure**: The institute has modern infrastructure and facili

In [5]:
class AnimeDataLoader:
    def __init__(self, original_csv: str, processed_csv: str):
        self.original_csv = original_csv
        self.processed_csv = processed_csv

    def load_and_process(self):
        df = pd.read_csv(
            self.original_csv,
            encoding="utf-8",
            on_bad_lines="skip"
        ).dropna()

        required_cols = {"Name", "Genres", "sypnopsis"}
        if not required_cols.issubset(df.columns):
            raise ValueError("Missing required columns in CSV")

        df["combined_info"] = (
            "Title: " + df["Name"]
            + " Overview: " + df["sypnopsis"]
            + " Genres: " + df["Genres"]
        )

        df[["combined_info"]].to_csv(
            self.processed_csv,
            index=False,
            encoding="utf-8"
        )

        return self.processed_csv

In [6]:
og_csv = "data/anime_with_synopsis.csv"
prossesed_csv = "data/anime_updated.csv"

data_loader = AnimeDataLoader(og_csv,prossesed_csv)

data_loader.load_and_process()

'data/anime_updated.csv'

In [1]:
19

19

DATA SPLITTING 

In [7]:
class VectorStoreBuilder:
    def __init__(self,csv_path : str , persist_dir : str = "chroma_db"):
        self.csv_path = csv_path
        self.persist_dir = persist_dir
        self.embedding = HuggingFaceEmbeddings(
            model_name="all-MiniLM-L6-v2"
        )

    def build_and_save_vectorstore(self):
        loader = CSVLoader(
            file_path=self.csv_path,
            encoding="utf-8",
            metadata_columns=[]
        
        )

        documents = loader.load()

        splitter = RecursiveCharacterTextSplitter(
            chunk_size = 1000,
            chunk_overlap = 20,
        )

        chunks = splitter.split_documents(documents)

        db = Chroma.from_documents(
            chunks ,
            self.embedding,
            persist_directory=self.persist_dir
        )

        db.persist()

    def load_vector_store(self):
        return Chroma(
            persist_directory=self.persist_dir,
            embedding_function=self.embedding
        )

In [8]:
print("BUILDING A VECTOR STORE")

og_csv = "data/anime_with_synopsis.csv"
prossesed_csv = "data/anime_updated.csv"
persist_dir = "chroma_db"

data_loader = AnimeDataLoader(og_csv,prossesed_csv)
processed_csv_path = data_loader.load_and_process()

BUILDING A VECTOR STORE


In [9]:
print("STILL BUILDING A VECTOR STORE")

vector_builder = VectorStoreBuilder(
    csv_path=processed_csv_path,
    persist_dir=persist_dir
)

vector_builder.build_and_save_vectorstore()

print("Vector database ready")

STILL BUILDING A VECTOR STORE


d:\nlp\Anime\anienv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Dell\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP d

Vector database ready


C:\Users\Dell\AppData\Local\Temp\ipykernel_8780\1358069252.py:32: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  db.persist()


RETRIVER TOOL

In [12]:
vector_builder = VectorStoreBuilder(
    csv_path = "",
    persist_dir="chroma_db"
)

retriever = vector_builder.load_vector_store().as_retriever()

In [14]:
query = "tell me about naruto"
retriever.invoke(query)

[Document(metadata={'row': 10, 'source': 'data/anime_updated.csv'}, page_content="combined_info: Title: Naruto Overview: oments prior to Naruto Uzumaki's birth, a huge demon known as the Kyuubi, the Nine-Tailed Fox, attacked Konohagakure, the Hidden Leaf Village, and wreaked havoc. In order to put an end to the Kyuubi's rampage, the leader of the village, the Fourth Hokage, sacrificed his life and sealed the monstrous beast inside the newborn Naruto. Now, Naruto is a hyperactive and knuckle-headed ninja still living in Konohagakure. Shunned because of the Kyuubi inside him, Naruto struggles to find his place in the village, while his burning desire to become the Hokage of Konohagakure leads him not only to some great new friends, but also some deadly foes. Genres: Action, Adventure, Comedy, Super Power, Martial Arts, Shounen"),
 Document(metadata={'source': 'data/anime_updated.csv', 'row': 43}, page_content='Kotori, the two sisters Mizukoshi Moe and Mako who prefer eating nabe on the s

In [15]:
@tool
def anime_retriever_tool(query: str) -> str:
    """
    Use this tool to search the anime knowledge base.

    Always call this tool for anime-related questions such as:
    recommendations, similarity search, genres, or plot summaries.

    Input:
    - query: User's anime preference or question.

    Output:
    - Relevant anime information retrieved from the vector database.
    """
    docs = retriever.invoke(query)
    return "\n\n".join(doc.page_content for doc in docs)


In [16]:
model_with_tools = model.bind_tools([anime_retriever_tool])

In [17]:
messages = [
    {
        "role": "user",
        "content":"can you suggest somethimg similar to naruto and also tell me a joke"
    }
]

In [18]:
ai_msg = model_with_tools.invoke(messages)
messages.append(ai_msg)

In [19]:
messages

[{'role': 'user',
  'content': 'can you suggest somethimg similar to naruto and also tell me a joke'},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '30g116tbg', 'function': {'arguments': '{"query":"anime similar to Naruto"}', 'name': 'anime_retriever_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 343, 'total_tokens': 385, 'completion_time': 0.096519941, 'completion_tokens_details': None, 'prompt_time': 0.017905971, 'prompt_tokens_details': None, 'queue_time': 0.058365199, 'total_time': 0.114425912}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c1354-4638-7051-822f-d953347a96be-0', tool_calls=[{'name': 'anime_retriever_tool', 'args': {'query': 'anime similar to Naruto'}, 'id': '30g116tbg', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'inpu

In [20]:
ai_msg.tool_calls

[{'name': 'anime_retriever_tool',
  'args': {'query': 'anime similar to Naruto'},
  'id': '30g116tbg',
  'type': 'tool_call'}]

In [21]:
messages[0]

{'role': 'user',
 'content': 'can you suggest somethimg similar to naruto and also tell me a joke'}

In [22]:
for tool_call in ai_msg.tool_calls:
    tool_result = anime_retriever_tool.invoke(tool_call)
    messages.append(tool_result)

In [23]:
messages

[{'role': 'user',
  'content': 'can you suggest somethimg similar to naruto and also tell me a joke'},
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '30g116tbg', 'function': {'arguments': '{"query":"anime similar to Naruto"}', 'name': 'anime_retriever_tool'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 343, 'total_tokens': 385, 'completion_time': 0.096519941, 'completion_tokens_details': None, 'prompt_time': 0.017905971, 'prompt_tokens_details': None, 'queue_time': 0.058365199, 'total_time': 0.114425912}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_dae98b5ecb', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019c1354-4638-7051-822f-d953347a96be-0', tool_calls=[{'name': 'anime_retriever_tool', 'args': {'query': 'anime similar to Naruto'}, 'id': '30g116tbg', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'inpu

In [25]:
final_response = model_with_tools.invoke(messages)
print(final_response.text)


Here's a joke: Why did the ninja go to the doctor? Because he was feeling a little "stealthy" and had a "killer" headache!

As for anime similar to Naruto, you might enjoy:

1. Ninin ga Shinobuden - a comedy anime about a girl who becomes friends with a ninja-in-training.
2. Tsubasa Chronicle - an adventure anime with a similar blend of action, fantasy, and romance.
3. Rurouni Kenshin - a historical action anime with a focus on samurai and ninja.

Let me know if you'd like more recommendations!
